# etape 1 : récupérer les ids des sites météos

In [28]:
# python
import unicodedata
import requests

BASE = "https://data.toulouse-metropole.fr/api/explore/v2.1/catalog/datasets/"

def normalize(s):
    if s is None:
        return ""
    s = str(s).lower()
    s = unicodedata.normalize("NFKD", s)
    return "".join(ch for ch in s if not unicodedata.combining(ch))

def _find_list_in_response(data):
    if isinstance(data, list) and all(isinstance(x, dict) for x in data):
        return data
    if isinstance(data, dict):
        for key in ("data", "datasets", "results", "records", "items", "entries"):
            v = data.get(key)
            if isinstance(v, list) and all(isinstance(x, dict) for x in v):
                return v
        # fallback: return the first list-of-dicts found among values
        for v in data.values():
            if isinstance(v, list) and all(isinstance(x, dict) for x in v):
                return v
    return None

def fetch_all_dataset_ids(limit=100, verbose=False):
    limit = min(max(1, int(limit)), 100)
    offset = 0
    ids = []
    session = requests.Session()

    while True:
        params = {"limit": limit, "offset": offset}
        r = session.get(BASE, params=params, timeout=10)
        r.raise_for_status()
        data = r.json()

        items = _find_list_in_response(data)
        if items is None:
            if verbose:
                snippet = str(data)
                print(f"Aucune liste détectée dans la réponse (status={r.status_code}). Extrait JSON:\n{snippet[:1000]}")
            break

        for it in items:
            if isinstance(it, dict):
                dsid = it.get("dataset_id") or it.get("id") or it.get("resource_id")
                if dsid:
                    ids.append(dsid)

        if len(items) < limit:
            break
        offset += limit

    return ids

# Exemple d'utilisation
if __name__ == "__main__":
    all_ids = fetch_all_dataset_ids(limit=100, verbose=True)
    print(f"Trouvés {len(all_ids)} ids")



Trouvés 793 ids


# Etape 2 : recupérer listing site

In [30]:
import unicodedata
import csv

def normalize(s):
    if s is None:
        return ""
    s = str(s).lower()
    s = unicodedata.normalize("NFKD", s)
    return "".join(ch for ch in s if not unicodedata.combining(ch))

def filter_meteo(ids):
    result = []
    for i in ids:
        n = normalize(i)
        if 'meteo' in n and 'archive' not in n:
            result.append(i)
    return result

try:
    ids_source = all_ids
except NameError:
    from __main__ import fetch_all_dataset_ids
    ids_source = fetch_all_dataset_ids(limit=100, verbose=False)

meteo_ids = filter_meteo(ids_source)

print(f"Trouvés {len(meteo_ids)} dataset_id contenant 'meteo'")
print("Exemples :", meteo_ids[:20])

out_path = "Data/meteo_ids.csv"
with open(out_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["dataset_id"])
    for d in meteo_ids:
        writer.writerow([d])
print(f"Sauvegardé dans `{out_path}`")

Trouvés 56 dataset_id contenant 'meteo'
Exemples : ['observation-meteorologique-historiques-france-synop', '14-station-meteo-toulouse-centre-pierre-potier', '12-station-meteo-toulouse-montaudran', '00-station-meteo-toulouse-valade', 'stations-meteo-en-place', '18-station-meteo-brax-ecole', '08-station-meteo-toulouse-basso-cambo', '09-station-meteo-toulouse-la-salade', '05-station-meteo-toulouse-nakache', '03-station-meteo-toulouse-busca', '07-station-meteo-toulouse-avenue-de-grande-bretagne', '01-station-meteo-toulouse-meteopole', '42-station-meteo-toulouse-parc-compans-cafarelli', '61-station-meteo-blagnac-mairie', '05-station-meteo-toulouse-life-hall-1', '32-station-meteo-mons-ecole', '34-station-meteo-toulouse-teso', '66-station-meteo-toulouse-life-coubertin', '25-station-meteo-tournefeuille-residentiel', '27-station-meteo-toulouse-saint-cyprien']
Sauvegardé dans `Data/meteo_ids.csv`


In [1]:
import pandas as pd
import requests
from datetime import datetime
from typing import Optional

class ExtractDataTimeT:
    """
    Classe pour extraire les données météo d'une station de Toulouse Métropole à l'instant T.
    Le fichier CSV doit contenir une seule colonne : 'dataset_id'.
    """

    def __init__(self, csv_path: str) -> None:
        """
        Initialise la classe avec le chemin du fichier CSV contenant les identifiants de datasets.

        Args:
            csv_path (str): Chemin vers le fichier CSV contenant une colonne 'dataset_id'.
        """
        self.stations_df: pd.DataFrame = pd.read_csv(csv_path)
        if 'dataset_id' not in self.stations_df.columns:
            raise ValueError("Le fichier CSV doit contenir une colonne 'dataset_id'.")
        
        #self.instant: instant_T = datetime.now().isoformat()
        self.dataset_id: Optional[str] = None
        self.data: Optional[dict] = None

    def choose_station(self) -> None:
        """
        Affiche la liste des identifiants de datasets et demande à l'utilisateur de choisir.
        Relance la saisie tant que l'entrée est invalide.
        """
        print("Stations disponibles :")
        for i, row in self.stations_df.iterrows():
            print(f"{i + 1}. {row['dataset_id']}")

        while True:
            try:
                choice: int = int(input("Choisissez une station (numéro) : ")) - 1
                if 0 <= choice < len(self.stations_df):
                    self.dataset_id = self.stations_df.loc[choice, 'dataset_id']
                    print(f"Dataset sélectionné : {self.dataset_id}")
                    break
                else:
                    print(f"Numéro invalide. Veuillez entrer un nombre entre 1 et {len(self.stations_df)}.")
            except ValueError:
                print("Entrée non valide. Veuillez entrer un numéro entier.")

    def fetch_data(self) -> None:
        """
        Interroge l'API pour récupérer les données météo du dataset sélectionné.
        """

        url: str = (
            f"https://data.toulouse-metropole.fr/api/explore/v2.1/catalog/datasets/"
            f"{self.dataset_id}/records?order_by=heure_de_paris%20DESC&limit=1"
        )
        try:
            response = requests.get(url)
            response.raise_for_status()
            self.data = response.json()
        except requests.RequestException as request_error:
            print(f"Erreur lors de la requête API : {request_error}")

    def to_dataframe(self) -> pd.DataFrame:
        """
        Convertit les données JSON en DataFrame pandas.

        Returns:
            pd.DataFrame: Tableau des résultats ou vide si aucune donnée.
        """
        if self.data and "results" in self.data:
            df = pd.DataFrame(self.data["results"])
            df["dataset_id"] = self.dataset_id  # Ajout de la colonne
            return df
        else:
            print("Aucune donnée disponible ou format inattendu.")
            return pd.DataFrame()


In [2]:
# Test class Exctractor
extractor = ExtractDataTimeT("data/meteo_ids.csv")
extractor.choose_station()
extractor.fetch_data()
df = extractor.to_dataframe()
print(df.head())

Stations disponibles :
1. observation-meteorologique-historiques-france-synop
2. 14-station-meteo-toulouse-centre-pierre-potier
3. 12-station-meteo-toulouse-montaudran
4. 00-station-meteo-toulouse-valade
5. stations-meteo-en-place
6. 18-station-meteo-brax-ecole
7. 08-station-meteo-toulouse-basso-cambo
8. 09-station-meteo-toulouse-la-salade
9. 05-station-meteo-toulouse-nakache
10. 03-station-meteo-toulouse-busca
11. 07-station-meteo-toulouse-avenue-de-grande-bretagne
12. 01-station-meteo-toulouse-meteopole
13. 42-station-meteo-toulouse-parc-compans-cafarelli
14. 61-station-meteo-blagnac-mairie
15. 05-station-meteo-toulouse-life-hall-1
16. 32-station-meteo-mons-ecole
17. 34-station-meteo-toulouse-teso
18. 66-station-meteo-toulouse-life-coubertin
19. 25-station-meteo-tournefeuille-residentiel
20. 27-station-meteo-toulouse-saint-cyprien
21. 33-station-meteo-saint-jory-chapelle-beldou
22. 26-station-meteo-toulouse-reynerie
23. 40-station-meteo-toulouse-zi-thibaud
24. 31-station-meteo-mons-s

In [ ]:
# transform

import pandas as pd
import requests
from datetime import datetime
from typing import Optional




In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 16 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   data                                        1 non-null      object 
 1   id                                          1 non-null      int64  
 2   humidite                                    1 non-null      int64  
 3   direction_du_vecteur_de_vent_max            0 non-null      object 
 4   pluie_intensite_max                         0 non-null      object 
 5   pression                                    0 non-null      object 
 6   direction_du_vecteur_vent_moyen             1 non-null      int64  
 7   type_de_station                             0 non-null      object 
 8   pluie                                       1 non-null      float64
 9   direction_du_vecteur_de_vent_max_en_degres  0 non-null      object 
 10  force_moyenne_du_v